In [ ]:
# -------------------------------------------------------
# pandas 라이브러리는 Python에서 데이터 분석과 조작을 손쉽게 해주는 핵심 라이브러리입니다.
# 엑셀, CSV 등 스프레드시트 파일을 읽고 쓰거나, 
# 데이터프레임(DataFrame) 형태로 로드하여 통계/그룹화/필터링 등을 직관적으로 할 수 있습니다.
# 본 프로젝트에서 sales.xlsx(엑셀 데이터)를 불러와서 
# 판매수량, 매출액 등을 Pandas로 집계할 때 사용합니다.
import pandas as pd

# -------------------------------------------------------
# os 모듈은 운영체제와 상호작용(예: 파일 경로, 디렉터리 관리)할 때 사용합니다.
# 예: os.path.exists()로 특정 파일이 존재하는지 확인하거나,
#     os.makedirs()로 폴더를 만들고, os.remove()로 파일을 삭제하는 등.
# 또한 업로드된 엑셀 파일을 서버 디렉터리(uploads) 안에 저장하거나
# 파일명/경로를 다루는 작업에서 유용하게 쓰입니다.
import os

In [ ]:
def update_stock(stock_file, sales_file):
    """
    재고 업데이트를 위한 함수(update_stock).

    1) stock_file(엑셀)에서 현재 재고(stock_df)를 불러옵니다.
    2) 만약 sales_file(엑셀)이 존재하지 않으면, 
       '판매 파일이 없다'고 안내하고 함수 실행을 종료합니다.
    3) 판매 데이터를 읽어 해당 product_id별 판매 개수(quantity_sold)를 집계한 뒤(sales_agg),
       재고 데이터(stock_df)와 병합(merge)하여 실제 판매량을 재고에 반영합니다.
    4) 판매량이 재고보다 많을 경우 재고를 0으로 처리(음수 재고 방지).
    5) 업데이트된 재고(merged_df["updated_stock"])를 원본 stock_df에 덮어쓰고, 
       다시 stock_file에 저장하여 재고 수량을 갱신합니다.

    :param stock_file: 재고가 들어 있는 엑셀 파일 경로 (str).
    :param sales_file: 판매 이력이 들어 있는 엑셀 파일 경로 (str).
    :return: None. 함수 실행 후 재고 파일(stock_file)이 업데이트됩니다.
    """

    # -------------------------------------------------------
    # stock_df: 기존 재고 정보를 담고 있는 데이터프레임.
    # 엑셀 파일 stock_file에서 읽어옴.
    stock_df = pd.read_excel(stock_file)

    # -------------------------------------------------------
    # 판매 파일(sales_file)이 없으면 재고 업데이트를 건너뜁니다.
    # 파일 유무를 os.path.exists()로 확인합니다.
    if not os.path.exists(sales_file):
        print(f"판매 파일({sales_file})이 없습니다. 재고 업데이트를 건너뜁니다.")
        return
    
    # -------------------------------------------------------
    # sales_df: 판매 이력이 담긴 데이터프레임.
    # 엑셀 파일 sales_file에서 읽어옴.
    sales_df = pd.read_excel(sales_file)
    
    # -------------------------------------------------------
    # product_id 컬럼별로 판매된 개수(quantity_sold)를 합산하여 
    # product_id별 total_sold(판매 총량) 컬럼을 생성한 데이터프레임(sales_agg)을 만듭니다.
    # groupby("product_id")로 제품 식별자별로 묶은 뒤 sum().
    # reset_index()로 그룹화 결과를 일반적인 형태로 되돌립니다.
    sales_agg = sales_df.groupby("product_id")["quantity_sold"].sum().reset_index()
    sales_agg.rename(columns={"quantity_sold": "total_sold"}, inplace=True)

    # -------------------------------------------------------
    # merge: 재고(stock_df)와 판매(sales_agg)를 product_id 기준으로 병합.
    # how="left"로, 재고에는 있지만 판매가 없었던 제품도 유지.
    # merged_df에는 total_sold 컬럼이 추가되어, 해당 product_id의 총 판매량이 들어감.
    merged_df = pd.merge(
        stock_df,
        sales_agg,
        on="product_id",
        how="left"
    )

    # -------------------------------------------------------
    # total_sold가 없는(=판매가 없었던) row는 NaN으로 되어 있을 것이므로 0으로 채웁니다.
    merged_df["total_sold"] = merged_df["total_sold"].fillna(0)

    # -------------------------------------------------------
    # updated_stock: 기존 재고(stock_qty)에서 total_sold(총 판매량)을 뺀 값.
    # 만약 음수가 되면 재고가 부족한 상태이므로 0으로 처리(재고는 음수가 될 수 없음).
    merged_df["updated_stock"] = merged_df["stock_qty"] - merged_df["total_sold"]
    merged_df.loc[merged_df["updated_stock"] < 0, "updated_stock"] = 0

    # -------------------------------------------------------
    # stock_df에 updated_stock 값을 반영:
    # (여기서는 merged_df의 updated_stock를 stock_df["stock_qty"]로 대체)
    stock_df["stock_qty"] = merged_df["updated_stock"]
    
    # -------------------------------------------------------
    # 최종적으로 수정된 재고 데이터를 엑셀 파일(stock_file)에 저장하여, 
    # 재고 데이터가 최신 상태가 되도록 합니다.
    stock_df.to_excel(stock_file, index=False)
    print("재고 업데이트 완료!")

In [ ]:
# -------------------------------------------------------
# update_stock 함수 호출 예시
# 
# stock_file(재고 엑셀)과 sales_file(판매 엑셀)의 절대경로를 지정하고,
# update_stock(...) 함수를 실행해 재고를 갱신합니다.
# 
# 1) stock.xlsx 파일에는 'product_id'와 'stock_qty' 컬럼이 존재해야 합니다.
# 2) sales.xlsx 파일에는 'product_id'와 'quantity_sold' 컬럼이 있어야 합니다.
# 3) 함수 실행 후, stock.xlsx는 'stock_qty' 컬럼이 판매량을 반영한 값으로 업데이트됩니다.
# 4) 만약 sales.xlsx가 없으면 재고 업데이트는 건너뛰며 안내 메시지만 출력됩니다.
# 5) 음수(재고 부족)로 계산된 항목은 0으로 조정됩니다.
# 
# 아래 경로들은 예시이므로, 실제 환경에 맞게 수정하세요.
# -------------------------------------------------------
update_stock(
    '/Users/hyeonuk/Documents/문서 - 김현욱의 Mac mini/Python/project/sample_data/stock.xlsx', 
    '/Users/hyeonuk/Documents/문서 - 김현욱의 Mac mini/Python/project/sample_data/sales.xlsx'
)

재고 업데이트 완료!
